In [1]:
!pip install sympy

In [2]:
# https://www.math.wsu.edu/students/odykhovychnyi/M201-04/Ch06_1-2_Simplex_Method.pdf

In [3]:
import numpy as np
from numpy import linalg as LA
from numpy.linalg import inv

In [4]:
# https://notebook.community/infimath/optimization-method/Simplex%20Tableau%20in%20Python

from __future__ import division
from numpy import *

# Ref: http://stackoverflow.com/questions/23344185/how-to-convert-a-decimal-number-into-fraction
from fractions import Fraction

class Tableau:
 
    def __init__(self, obj):
        self.obj = [1] + obj
        self.rows = []
        self.cons = []
        self.no_variables = len(obj)
        self.no_constraints = 0
        self.is_fraction = False # set True to output in fraction
 
    def add_constraint(self, expression, value):
        self.rows.append([0] + expression)
        self.cons.append(value)
        self.no_constraints += 1
        self.header_tableau = ["Basic"] + ["x"+str(i+1) for i in range(self.no_variables)] \
                                        + ["s"+str(i+1) for i in range(self.no_constraints)] \
                                        + ["Solution"]
                
        self.basic_variables = ["s"+str(i+1) for i in range(self.no_constraints)]
 
    def _pivot_column(self):
        low = 0
        idx = 0
        for i in range(1, len(self.obj)-1):
            if self.obj[i] < low:
                low = self.obj[i]
                idx = i
        if idx == 0: return -1
        return idx
 
    def _pivot_row(self, col):
        rhs = [self.rows[i][-1] for i in range(len(self.rows))]
        lhs = [self.rows[i][col] for i in range(len(self.rows))]
        ratio = []
        for i in range(len(rhs)):
            if lhs[i] == 0:
                ratio.append(99999999 * abs(max(rhs)))
                continue
            ratio.append(rhs[i]/lhs[i])
        return argmin(ratio)
 
    def display(self):   
        if self.is_fraction:
            # Formatting the output in fraction
            # Ref: https://pyformat.info/
            fmt = '{:<8}'.format("Basic") \
                  + "".join(['{:>8}'.format("x"+str(i+1)) for i in range(self.no_variables)])   \
                  + "".join(['{:>8}'.format("s"+str(i+1)) for i in range(self.no_constraints)]) \
                  + '{:>8}'.format("Sol.")

            fmt += "\n" 
            fmt += '{:<8}'.format("z") \
                   + "".join(["{:>8}".format(item) for item in self.obj[1:]])

            for i, row in enumerate(self.rows):
                fmt += "\n" 
                fmt += '{:<8}'.format(self.basic_variables[i]) \
                       + "".join(["{:>8}".format(item) for item in row[1:]])
            print(fmt)
            
        else:
            # Formatting the output in float with 2 decimal places
            fmt = '{:<8}'.format("Basic") \
                  + "".join(['{:>8}'.format("x"+str(i+1)) for i in range(self.no_variables)])   \
                  + "".join(['{:>8}'.format("s"+str(i+1)) for i in range(self.no_constraints)]) \
                  + '{:>8}'.format("Sol.")

            fmt += "\n" 
            fmt += '{:<8}'.format("z") + "".join(["{:>8.2f}".format(item) for item in self.obj[1:]])

            for i, row in enumerate(self.rows):
                fmt += "\n" 
                fmt += '{:<8}'.format(self.basic_variables[i]) \
                       + "".join(["{:>8.2f}".format(item) for item in row[1:]])
            print(fmt)            
              
#       print '\n', matrix([self.obj] + self.rows)
 
    def _pivot(self, row, col):
        e = self.rows[row][col]
        self.rows[row] /= e
        for r in range(len(self.rows)):
            if r == row: continue
            self.rows[r] = self.rows[r] - self.rows[r][col]*self.rows[row]
        self.obj = self.obj - self.obj[col]*self.rows[row]
 
    def _check(self):
        if min(self.obj[1:-1]) >= 0: return 1
        return 0
         
    def solve(self):
        # build full tableau
        for i in range(len(self.rows)):
            self.obj += [0]
            ident = [0 for r in range(len(self.rows))]
            ident[i] = 1
            self.rows[i] += ident + [self.cons[i]]
            self.rows[i] = array(self.rows[i], dtype=float)
        self.obj = array(self.obj + [0], dtype=float)
 
        # solve
        self.display()
        while not self._check():
            c = self._pivot_column()
            r = self._pivot_row(c)
            self._pivot(r,c)
            # print '\npivot column: %s\npivot row: %s'%(c+1,r+2)
            print('\n')
            print('Entering Variable: ', self.header_tableau[c])
            print('Leaving Variable : ', self.basic_variables[r])
            print('\n')
            # Updating the basic variable
            for index, item in enumerate(self.basic_variables):
                if self.basic_variables[index] == self.basic_variables[r]:
                    self.basic_variables[index] = self.header_tableau[c]
                               
            self.display()

t = Tableau([-2,-3,-2])
t.add_constraint([2, 1, 1], 4)
t.add_constraint([1, 2, 1], 7)
t.add_constraint([0, 0, 1], 5)
t.is_fraction = True
t.solve()

Basic         x1      x2      x3      s1      s2      s3    Sol.
z           -2.0    -3.0    -2.0     0.0     0.0     0.0     0.0
s1           2.0     1.0     1.0     1.0     0.0     0.0     4.0
s2           1.0     2.0     1.0     0.0     1.0     0.0     7.0
s3           0.0     0.0     1.0     0.0     0.0     1.0     5.0


Entering Variable:  x2
Leaving Variable :  s2


Basic         x1      x2      x3      s1      s2      s3    Sol.
z           -0.5     0.0    -0.5     0.0     1.5     0.0    10.5
s1           1.5     0.0     0.5     1.0    -0.5     0.0     0.5
x2           0.5     1.0     0.5     0.0     0.5     0.0     3.5
s3           0.0     0.0     1.0     0.0     0.0     1.0     5.0


Entering Variable:  x1
Leaving Variable :  s1


Basic         x1      x2      x3      s1      s2      s3    Sol.
z            0.0     0.0-0.333333333333333370.33333333333333331.3333333333333333     0.010.666666666666666
x1           1.0     0.00.33333333333333330.6666666666666666-0.333333333333333

In [5]:
# https://github.com/MichaelStott/SimplexSolver

import ast, getopt, sys, copy, os
from fractions import Fraction

clear = lambda: os.system('cls' if os.name == 'nt' else 'clear')


class SimplexSolver():
    ''' Solves linear programs using simplex algorithm and
        output problem steps in LaTeX file.
    '''

    # Table for converting inequality list to LaTeX    
    latex_ineq = {'=': '=',
                  '<=': r'\leq',
                  '>=': r'\geq'}

    def __init__(self):
        self.A = []
        self.b = []
        self.c = []
        self.tableau = []
        self.entering = []
        self.departing = []
        self.ineq = []
        self.prob = "max"
        self.gen_doc = False
        self.doc = ""

    def run_simplex(self, A, b, c, prob='max', ineq=[],
                    enable_msg=False, latex=False):
        ''' Run simplex algorithm.
        '''
        self.prob = prob
        self.gen_doc = latex
        self.ineq = ineq

        # Create the header for the latex doc.        
        self.start_doc()

        # Add slack & artificial variables
        self.set_simplex_input(A, b, c)
            
        # Are there any negative elements on the bottom (disregarding
        # right-most element...)
        while (not self.should_terminate()):
            # ... if so, continue.
            if(enable_msg):
                clear()
                self._print_tableau()
                print(("Current solution: %s\n" %
                      str(self.get_current_solution())))
                self._prompt()
            
            # Attempt to find a non-negative pivot.
            pivot = self.find_pivot()
            if pivot[1] < 0:
                if (enable_msg):
                    print ("There exists no non-negative pivot. "
                           "Thus, the solution is infeasible.")
                self.infeasible_doc()
                self.print_doc()
                return None
            else:
                self.pivot_doc(pivot)
                if (enable_msg):
                    clear()
                    self._print_tableau()
                    print(("\nThere are negative elements in the bottom row, "
                          "so the current solution is not optimal. "
                          "Thus, pivot to improve the current solution. The "
                          "entering variable is %s and the departing "
                          "variable is %s.\n" %
                           (str(self.entering[pivot[0]]),
                           str(self.departing[pivot[1]]))))
                    self._prompt()
                    print("\nPerform elementary row operations until the "
                          "pivot is one and all other elements in the "
                          "entering column are zero.\n")

            # Do row operations to make every other element in column zero.
            self.pivot(pivot)
            self.tableau_doc()

        solution = self.get_current_solution()
        self.final_solution_doc(solution)
        if (enable_msg):
            clear()
            self._print_tableau()
            print(("Current solution: %s\n" % str(solution)))
            print("That's all folks!")
        self.print_doc()
        return solution
        
    def set_simplex_input(self, A, b, c):
        ''' Set initial variables and create tableau.
        '''
        # Convert all entries to fractions for readability.
        for a in A:
            self.A.append([Fraction(x) for x in a])    
        self.b = [Fraction(x) for x in b]
        self.c = [Fraction(x) for x in c]
        if not self.ineq:
            if self.prob == 'max':
                self.ineq = ['<='] * len(b)
            elif self.prob == 'min':
                self.ineq = ['>='] * len(b)
            
        self.update_enter_depart(self.get_Ab())
        self.init_problem_doc()

        # If this is a minimization problem...
        if self.prob == 'min':
            # ... find the dual maximum and solve that.
            m = self.get_Ab()
            m.append(self.c + [0])
            m = [list(t) for t in zip(*m)] # Calculates the transpose
            self.A = [x[:(len(x)-1)] for x in m]
            self.b = [y[len(y) - 1] for y in m]
            self.c = m[len(m) -1]
            self.A.pop()
            self.b.pop()
            self.c.pop()
            self.ineq = ['<='] * len(self.b)

        self.create_tableau()
        self.ineq = ['='] * len(self.b)
        self.update_enter_depart(self.tableau)
        self.slack_doc()
        self.init_tableau_doc()

    def update_enter_depart(self, matrix):
        self.entering = []
        self.departing = []
        # Create tables for entering and departing variables
        for i in range(0, len(matrix[0])):
            if i < len(self.A[0]):
                prefix = 'x' if self.prob == 'max' else 'y'
                self.entering.append("%s_%s" % (prefix, str(i + 1)))
            elif i < len(matrix[0]) - 1:
                self.entering.append("s_%s" % str(i + 1 - len(self.A[0])))
                self.departing.append("s_%s" % str(i + 1 - len(self.A[0])))
            else:
                self.entering.append("b")

    def add_slack_variables(self):
        ''' Add slack & artificial variables to matrix A to transform
            all inequalities to equalities.
        '''
        slack_vars = self._generate_identity(len(self.tableau))
        for i in range(0, len(slack_vars)):
            self.tableau[i] += slack_vars[i]
            self.tableau[i] += [self.b[i]]

    def create_tableau(self):
        ''' Create initial tableau table.
        '''
        self.tableau = copy.deepcopy(self.A)
        self.add_slack_variables()
        c = copy.deepcopy(self.c)
        for index, value in enumerate(c):
            c[index] = -value
        self.tableau.append(c + [0] * (len(self.b)+1))

    def find_pivot(self):
        ''' Find pivot index.
        '''
        enter_index = self.get_entering_var()
        depart_index = self.get_departing_var(enter_index)
        return [enter_index, depart_index]

    def pivot(self, pivot_index):
        ''' Perform operations on pivot.
        '''
        j,i = pivot_index

        pivot = self.tableau[i][j]
        self.tableau[i] = [element / pivot for
                           element in self.tableau[i]]
        for index, row in enumerate(self.tableau):
            if index != i:
                row_scale = [y * self.tableau[index][j]
                             for y in self.tableau[i]]
                self.tableau[index] = [x - y for x,y in
                                       zip(self.tableau[index],
                                       row_scale)]

        self.departing[i] = self.entering[j]
        
    def get_entering_var(self):
        ''' Get entering variable by determining the 'most negative'
            element of the bottom row.
        '''
        bottom_row = self.tableau[len(self.tableau) - 1]
        most_neg_ind = 0
        most_neg = bottom_row[most_neg_ind]
        for index, value in enumerate(bottom_row):
            if value < most_neg:
                most_neg = value
                most_neg_ind = index
        return most_neg_ind
            

    def get_departing_var(self, entering_index):
        ''' To calculate the departing variable, get the minimum of the ratio
            of b (b_i) to the corresponding value in the entering collumn. 
        '''
        skip = 0
        min_ratio_index = -1
        min_ratio = 0
        for index, x in enumerate(self.tableau):
            if x[entering_index] != 0 and x[len(x)-1]/x[entering_index] > 0:
                skip = index
                min_ratio_index = index
                min_ratio = x[len(x)-1]/x[entering_index]
                break
        
        if min_ratio > 0:
            for index, x in enumerate(self.tableau):
                if index > skip and x[entering_index] > 0:
                    ratio = x[len(x)-1]/x[entering_index]
                    if min_ratio > ratio:
                        min_ratio = ratio
                        min_ratio_index = index
        
        return min_ratio_index

    def get_Ab(self):
        ''' Get A matrix with b vector appended.
        '''
        matrix = copy.deepcopy(self.A)
        for i in range(0, len(matrix)):
            matrix[i] += [self.b[i]]
        return matrix

    def should_terminate(self):
        ''' Determines whether there are any negative elements
            on the bottom row
        '''
        result = True
        index = len(self.tableau) - 1
        for i, x in enumerate(self.tableau[index]):
            if x < 0 and i != len(self.tableau[index]) - 1:
                result = False
        return result

    def get_current_solution(self):
        ''' Get the current solution from tableau.
        '''
        solution = {}
        for x in self.entering:
            if x != 'b':
                if x in self.departing:
                    solution[x] = self.tableau[self.departing.index(x)]\
                                  [len(self.tableau[self.departing.index(x)])-1]
                else:
                    solution[x] = 0
        solution['z'] = self.tableau[len(self.tableau) - 1]\
                          [len(self.tableau[0]) - 1]
        
        # If this is a minimization problem...
        if (self.prob == 'min'):
            # ... then get x_1, ..., x_n  from last element of
            # the slack columns.
            bottom_row = self.tableau[len(self.tableau) - 1]
            for v in self.entering:
                if 's' in v:
                    solution[v.replace('s', 'x')] = bottom_row[self.entering.index(v)]    

        return solution

    def start_doc(self):
        if not self.gen_doc:
            return
        self.doc = (r"\documentclass{article}"
                    r"\usepackage{amsmath}"
                    r"\begin{document}"
                    r"\title{Simplex Solver}"
                    r"\maketitle"
                    r"\begin{flushleft}"
                    r"\textbf{Problem}"
                    r"\end{flushleft}")

    def init_problem_doc(self):
        if not self.gen_doc:
            return
        # Objective function.
        self.doc += (r"\begin{flushleft}"
                     r"Given the following linear system and objective "
                     r"function, find the optimal solution."
                     r"\end{flushleft}"
                     r"\begin{equation*}")
        func = ""
        found_value = False
        for index, x in enumerate(self.c):
            opp = '+'
            if x == 0:
                continue
            if x < 0:
                opp = ' - '
            elif index == 0 or not found_value:
                opp = ''
            if x == 1 or x == -1:
                x = ''
            func += (r"%s %sx_%s "  % (opp, str(x), str(index+1)))
            found_value = True
        self.doc += (r"\max{%s} \\ "
                     r"\end{equation*}" % func)
        self.linear_system_doc(self.get_Ab())
        self.doc += (r"\begin{flushleft}"
                     r"\textbf{Solution}"
                     r"\end{flushleft}")

    def linear_system_doc(self, matrix):
        if not self.gen_doc:
            return
        self.doc += (r"\["
                     r"\left\{"
                     r"\begin{array}{c}")
        for i in range(0, len(matrix)):
            found_value = False
            for index, x in enumerate(matrix[i]):
                opp = '+'
                if x == 0 and index != len(matrix[i]) - 1:
                    continue
                if x < 0:
                    opp = '-'
                elif index == 0 or not found_value:
                    opp = ''
                if index != len(matrix[i]) - 1:
                    if x == 1 or x == -1:
                        x = ''
                    self.doc += (r"%s %s%s "  % (opp, str(x),
                                                 str(self.entering[index])))
                else:
                    self.doc += (r"%s %s"  % (self.latex_ineq[self.ineq[i]],str(x)))
                found_value = True
                if (index == len(matrix[i]) - 1):
                    self.doc += r" \\ "        
        self.doc += (r"\end{array}"
                     r"\right."
                     r"\]")
 
    def slack_doc(self):
        if not self.gen_doc:
            return
        self.doc += (r"\begin{flushleft}"
                     r"Add slack variables to turn "
                     r"all inequalities to equalities."
                     r"\end{flushleft}")
        self.linear_system_doc(self.tableau[:len(self.tableau)-1])

    def init_tableau_doc(self):
        if not self.gen_doc:
            return
        self.doc += (r"\begin{flushleft}"
                     r"Create the initial tableau of the new linear system."
                     r"\end{flushleft}")
        self.tableau_doc()
            
    def tableau_doc(self):
        if not self.gen_doc:
            return
        self.doc += r"\begin{equation*}"
        self.doc += r"\begin{bmatrix}"
        self.doc += r"\begin{array}{%s|c}" % ("c" * (len(self.tableau[0])-1))
        for index, var in enumerate(self.entering):
            if index != len(self.entering) - 1:
                self.doc += r"%s &" % var
            else:
                self.doc += r"%s \\ \hline" % var
        for indexr, row in enumerate(self.tableau):
            for indexv, value in enumerate(row):
                if indexv != (len(row)-1):
                    self.doc += r"%s & " % (str(value))
                elif indexr != (len(self.tableau)-2):
                    self.doc += r"%s \\" % (str(value))
                else:
                    self.doc += r"%s \\ \hline" % (str(value))
        self.doc += r"\end{array}"
        self.doc += r"\end{bmatrix}"
        self.doc += (r"\begin{array}{c}"
                     r"\\")
        for var in self.departing:
            self.doc += (r"%s \\" % var)
        self.doc += r"\\"
        self.doc += r"\end{array}"
        self.doc += r"\end{equation*}"

    def infeasible_doc(self):
        if not self.gen_doc:
            return
        self.doc += (r"\begin{flushleft}"
                     r"There are no non-negative candidates for the pivot. "
                     r"Thus, the solution is infeasible."
                     r"\end{flushleft}")

    def pivot_doc(self, pivot):
        if not self.gen_doc:
            return
        self.doc += (r"\begin{flushleft}"
                     r"There are negative elements in the bottom row, "
                     r"so the current solution is not optimal. "
                     r"Thus, pivot to improve the current solution. The "
                     r"entering variable is $%s$ and the departing "
                     r"variable is $%s$."
                     r"\end{flushleft}" %
                     (str(self.entering[pivot[0]]),
                     str(self.departing[pivot[1]])))
        self.doc += (r"\begin{flushleft}"
                     r"Perform elementary row operations until the "
                     r"pivot element is 1 and all other elements in the "
                     r"entering column are 0."
                     r"\end{flushleft}")
    
    def current_solution_doc(self, solution):
        if not self.gen_doc:
            return
        self.doc += r"\begin{equation*}"
        for key,value in sorted(solution.items()):
            self.doc += r"%s = %s" % (key, self._fraction_to_latex(value))
            if key != 'z':
                self.doc += r", "
        self.doc += r"\end{equation*}"

    def final_solution_doc(self, solution):
        if not self.gen_doc:
            return
        self.doc += (r"\begin{flushleft}"
                     r"There are no negative elements in the bottom row, so "
                     r"we know the solution is optimal. Thus, the solution is: "
                     r"\end{flushleft}")
        self.current_solution_doc(solution)

    def print_doc(self):
        if not self.gen_doc:
            return
        self.doc += (r"\end{document}")
        with open("solution.tex", "w") as tex:
            tex.write(self.doc)

    def _fraction_to_latex(self, fract):
        if fract.denominator == 1:
            return str(fract.numerator)
        else:
            return r"\frac{%s}{%s}" % (str(fract.numerator), str(fract.denominator))

    def _generate_identity(self, n):
        ''' Helper function for generating a square identity matrix.
        '''
        I = []
        for i in range(0, n):
            row = []
            for j in range(0, n):
                if i == j:
                    row.append(1)
                else:
                    row.append(0)
            I.append(row)
        return I
        
    def _print_matrix(self, M):
        ''' Print some matrix.
        '''
        for row in M:
            print('|', end=' ')
            for val in row:
                print('{:^5}'.format(str(val)), end=' ')
            print('|')

    def _print_tableau(self):
        ''' Print simplex tableau.
        '''
        print(' ', end=' ')
        for val in self.entering:
            print('{:^5}'.format(str(val)), end=' ')
        print(' ')
        for num, row in enumerate(self.tableau):
            print('|', end=' ')
            for index, val in enumerate(row):
                print('{:^5}'.format(str(val)), end=' ')
            if num < (len(self.tableau) -1):
                print('| %s' % self.departing[num])
            else:
                print('|')

    def _prompt(self):
        input("Press enter to continue...")

'''
if __name__ == '__main__':
    clear()

    # COMMAND LINE INPUT HANDLING
    A = []
    b = []
    c = []
    p = ''
    argv = sys.argv[1:]    
    try:
        opts, args = getopt.getopt(argv,"hA:b:c:p:",["A=","b=","c=","p="])
    except getopt.GetoptError:
        print('simplex.py -A <matrix> -b <vector> -c <vector> -p <type>')
        sys.exit(2)
    for opt, arg in opts:
        if opt == '-h':
            print('simplex.py -A <matrix> -b <vector> -c <vector> -p <obj_func_type>')
            print('A: Matrix that represents coefficients of constraints.')
            print('b: Ax <= b')
            print('c: Coefficients of objective function.')
            print('p: Indicates max or min objective function.')
            sys.exit()
        elif opt in ("-A"):
            A = ast.literal_eval(arg)
        elif opt in ("-b"):
            b = ast.literal_eval(arg)
        elif opt in ("-c"):
            c = ast.literal_eval(arg)
        elif opt in ("-p"):
            p = arg.strip()
    if not A or not b or not c:
        print('Must provide arguments for A, b, c (use -h for more info)')
        sys.exit()
    # END OF COMMAND LINE INPUT HANDLING

    # Assume maximization problem as default.
    if p not in ('max', 'min'):
        p = 'max'
    
    SimplexSolver().run_simplex(A,b,c,prob=p,enable_msg=False,latex=True)
'''

A = [[-2.0, -1.0], [-1.0, -2.0]]
b = [-3.0, -3.0]
c = [-4.0, -5.0]
p = 'min'

SimplexSolver().run_simplex(A,b,c,prob=p)

{'y_1': 0,
 'y_2': 0,
 's_1': Fraction(-4, 1),
 's_2': Fraction(-5, 1),
 'z': 0,
 'x_1': 0,
 'x_2': 0}

In [6]:
# https://medium.com/@jacob.d.moore1/coding-the-simplex-algorithm-from-scratch-using-python-and-numpy-93e3813e6e70

import numpy as np

def gen_matrix(var,cons):    
    tab = np.zeros((cons+1, var+cons+2))    
    return tab

def next_round_r(table):    
    m = min(table[:-1,-1])    
    if m>= 0:        
        return False    
    else:        
        return True

def next_round(table):    
    lr = len(table[:,0])   
    m = min(table[lr-1,:-1])    
    if m>=0:
        return False
    else:
        return True

def find_neg_r(table):
    lc = len(table[0,:])
    m = min(table[:-1,lc-1])
    if m<=0:        
        n = np.where(table[:-1,lc-1] == m)[0][0]
    else:
        n = None
    return n

def find_neg(table):
    lr = len(table[:,0])
    m = min(table[lr-1,:-1])
    if m<=0:
        n = np.where(table[lr-1,:-1] == m)[0][0]
    else:
        n = None
    return n

def loc_piv_r(table):
    total = []        
    r = find_neg_r(table)
    row = table[r,:-1]
    m = min(row)
    c = np.where(row == m)[0][0]
    col = table[:-1,c]
    for i, b in zip(col,table[:-1,-1]):
        if i**2>0 and b/i>0:
            total.append(b/i)
        else:                
            total.append(10000)
    index = total.index(min(total))        
    return [index,c]

def loc_piv(table):
    if next_round(table):
        total = []
        n = find_neg(table)
        for i,b in zip(table[:-1,n],table[:-1,-1]):
            if i**2>0 and b/i >0:
                total.append(b/i)
            else:
                total.append(10000)
        index = total.index(min(total))
        return [index,n]

def pivot(row,col,table):
    lr = len(table[:,0])
    lc = len(table[0,:])
    t = np.zeros((lr,lc))
    pr = table[row,:]
    if table[row,col]**2>0:
        e = 1/table[row,col]
        r = pr*e
        for i in range(len(table[:,col])):
            k = table[i,:]
            c = table[i,col]
            if list(k) == list(pr):
                continue
            else:
                t[i,:] = list(k-r*c)
        t[row,:] = list(r)
        return t
    else:
        print('Cannot pivot on this element.')

def convert(eq):
    eq = eq.split(',')
    if 'G' in eq:
        g = eq.index('G')
        del eq[g]
        eq = [float(i)*-1 for i in eq]
        return eq
    if 'L' in eq:
        l = eq.index('L')
        del eq[l]
        eq = [float(i) for i in eq]
        return eq

def convert_min(table):
    table[-1,:-2] = [-1*i for i in table[-1,:-2]]
    table[-1,-1] = -1*table[-1,-1]    
    return table

def gen_var(table):
    lc = len(table[0,:])
    lr = len(table[:,0])
    var = lc - lr -1
    v = []
    for i in range(var):
        v.append('x'+str(i+1))
    return v

def add_cons(table):
    lr = len(table[:,0])
    empty = []
    for i in range(lr):
        total = 0
        for j in table[i,:]:                       
            total += j**2
        if total == 0: 
            empty.append(total)
    if len(empty)>1:
        return True
    else:
        return False

def constrain(table,eq):
    if add_cons(table) == True:
        lc = len(table[0,:])
        lr = len(table[:,0])
        var = lc - lr -1      
        j = 0
        while j < lr:            
            row_check = table[j,:]
            total = 0
            for i in row_check:
                total += float(i**2)
            if total == 0:                
                row = row_check
                break
            j +=1
        eq = convert(eq)
        i = 0
        while i<len(eq)-1:
            row[i] = eq[i]
            i +=1        
        row[-1] = eq[-1]
        row[var+j] = 1    
    else:
        print('Cannot add another constraint.')

def add_obj(table):
    lr = len(table[:,0])
    empty = []
    for i in range(lr):
        total = 0        
        for j in table[i,:]:
            total += j**2
        if total == 0:
            empty.append(total)    
    if len(empty)==1:
        return True
    else:
        return False

def obj(table,eq):
    if add_obj(table)==True:
        eq = [float(i) for i in eq.split(',')]
        lr = len(table[:,0])
        row = table[lr-1,:]
        i = 0        
        while i<len(eq)-1:
            row[i] = eq[i]*-1
            i +=1
        row[-2] = 1
        row[-1] = eq[-1]
    else:
        print('You must finish adding constraints before the objective function can be added.')

def maxz(table):
    while next_round_r(table)==True:
        table = pivot(loc_piv_r(table)[0],loc_piv_r(table)[1],table)
    while next_round(table)==True:
        table = pivot(loc_piv(table)[0],loc_piv(table)[1],table)        
    lc = len(table[0,:])
    lr = len(table[:,0])
    var = lc - lr -1
    i = 0
    val = {}
    for i in range(var):
        col = table[:,i]
        s = sum(col)
        m = max(col)
        if float(s) == float(m):
            loc = np.where(col == m)[0][0]            
            val[gen_var(table)[i]] = table[loc,-1]
        else:
            val[gen_var(table)[i]] = 0
    val['max'] = table[-1,-1]
    return val

def minz(table):
    table = convert_min(table)
    while next_round_r(table)==True:
        table = pivot(loc_piv_r(table)[0],loc_piv_r(table)[1],table)    
    while next_round(table)==True:
        table = pivot(loc_piv(table)[0],loc_piv(table)[1],table)       
    lc = len(table[0,:])
    lr = len(table[:,0])
    var = lc - lr -1
    i = 0
    val = {}
    for i in range(var):
        col = table[:,i]
        s = sum(col)
        m = max(col)
        if float(s) == float(m):
            loc = np.where(col == m)[0][0]             
            val[gen_var(table)[i]] = table[loc,-1]
        else:
            val[gen_var(table)[i]] = 0 
            val['min'] = table[-1,-1]*-1
    return val

In [7]:
m = gen_matrix(2, 2)
constrain(m,'2,1,L,3')
constrain(m,'1,2,L,3')
obj(m,'-4,-5,0')
print(minz(m))

{'x1': 1.0, 'x2': 1.0}


In [8]:
m = gen_matrix(2, 4)
constrain(m,'2,1,L,3')
constrain(m,'1,2,L,3')
constrain(m,'1,0,L,0')
constrain(m,'-1,0,L,0')
obj(m,'-4,-5,0')
print(minz(m))

{'x1': 1.0, 'x2': 1.0}


In [9]:
m = gen_matrix(2, 3)
constrain(m,'1,1,L,1')
constrain(m,'0,1,L,0')
constrain(m,'0,-1,L,0')
obj(m,'-1,-1,0')
print(minz(m))

{'x1': 1.0, 'x2': 1.0}


In [10]:
#simple lp

import numpy as np
import scipy.sparse as sparse
import scipy.sparse.linalg as linalg
# min cx
# x >= 0
# Ax = b

def newtonDec(df, dx):
    return np.dot(df,dx)

# assumes that x + alpha*dx can be made positive
def linesearch(x, dx):
    alpha = 1.
    while not np.all( x + alpha*dx > 0):
        alpha *= 0.1
    return alpha

# min cx

def solve_lp2(A, b, c, gamma, xstart=None):
    #x = np.ones(A.shape[1])
    #lam = np.zeros(b.shape)
    xsize = A.shape[1]
    if xstart is not None:
        x = xstart
    else:
        #xlam = np.ones(xsize + b.size)
        x = np.ones(xsize) # xlam[:xsize]
        #lam = xlam[xsize:]
    while True :
        ##print("Iterate")
        H = sparse.bmat( [[ sparse.diags(gamma / x**2)   ,   A.T ],
                          [ A  ,                         0 ]]  )

        dfdx = c - gamma / x #+  A.T@lam 
        dfdlam = A@x - b
        df = np.concatenate((dfdx, dfdlam))#np.zeros(b.size))) # dfdlam))
        #np.concatenate( , A@x - b)
        dxlam = linalg.spsolve(H,df)
        dx = - dxlam[:xsize]
        lam = dxlam[xsize:]

        alpha = linesearch(x,dx)
        x += alpha * dx
        #lam += dlam
        if newtonDec(dfdx,dx) >= -1e-10:
            ##print("stop")
            break

    return x, lam


def solve_lp(A,b,c, xstart=None):
    gamma = 1.0
    xsize = A.shape[1]
    x = np.ones(xsize)
    for i in range(8):
        x, lam = solve_lp2(A, b, c, gamma, xstart=x)
        gamma *= 0.1
    return x, lam


N = 3
A = np.ones(N).reshape(1,-1)
b = np.ones(1)*2
c = np.zeros(N)
c[0] = -1
print(solve_lp(A,b,c))

(array([1.9999998e+00, 9.9999995e-08, 9.9999995e-08]), array([-1.00000005]))


D:\anaconda3\envs\lets-plot-experiments\lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:214: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',


In [11]:
A = np.array([[1, -2]])
b = np.array([0])
c = np.array([2, 1])
print(solve_lp(A,b,c))

(array([8.e-08, 4.e-08]), array([0.75]))


In [12]:
def lp_solver_custom(c1, A1, b1):
    import numpy as np
    import scipy.sparse as sparse
    import scipy.sparse.linalg as linalg

    def newtonDec(df, dx):
        return np.dot(df,dx)

    def linesearch(x, dx):
        alpha = 1.
        while not np.all( x + alpha*dx > 0):
            alpha *= 0.1
        return alpha

    def solve_lp2(A, b, c, gamma, xstart=None):
        xsize = A.shape[1]
        if xstart is not None:
            x = xstart
            print("x")
            print(x)
            print(A.shape)
        else:
            x = np.ones(xsize)
        while True :
            H = sparse.bmat( [[ sparse.diags(gamma / x**2)   ,   A.T ],
                              [ A  ,                         np.zeros((A.shape[0], A.shape[0])) ]]  )

            dfdx = c - gamma / x
            dfdlam = A@x - b
            df = np.concatenate((dfdx, dfdlam))
            dxlam = linalg.spsolve(H,df)
            dx = - dxlam[:xsize]
            lam = dxlam[xsize:]

            alpha = linesearch(x,dx)
            x += alpha * dx
            if newtonDec(dfdx,dx) >= -1e-10:
                break

        return x, lam

    def solve_lp(A,b,c, xstart=None):
        gamma = 1.0
        xsize = A.shape[1]
        x = np.ones(xsize)
        for i in range(8):
            x, lam = solve_lp2(A, b, c, gamma, xstart=x)
            gamma *= 0.1
        return x, lam

    return solve_lp(A1, b1, c1)[0]

A2 = np.array([[1, -1, 0]])
b2 = np.array([1])
c2 = np.array([1, -1, 1])
lp_solver_custom(c2, A2, b2)

x
[1. 1. 1.]
(1, 3)
x
[3.15111918e+16 3.15111918e+16 1.00000000e+00]
(1, 3)
x
[3.15111918e+16 3.15111918e+16 1.00000000e-01]
(1, 3)
x
[3.15111918e+16 3.15111918e+16 1.00000000e-02]
(1, 3)
x
[3.15111918e+16 3.15111918e+16 1.00000000e-03]
(1, 3)
x
[3.15111918e+16 3.15111918e+16 1.00000000e-04]
(1, 3)
x
[3.15111918e+16 3.15111918e+16 1.00000000e-05]
(1, 3)
x
[3.15111918e+16 3.15111918e+16 1.00000000e-06]
(1, 3)


array([3.15111918e+16, 3.15111918e+16, 1.00000000e-07])

In [13]:
def lp_solver_custom2(c1, A1, b1):
    import numpy as np
    import sympy
    from numpy.linalg import matrix_rank


    class InteriorPointSolver:
        """ This class implements primal-dual (infeasible) interior-point method to solve LPs """

        def solve(self, c, A, b, epsilon=0.0001):
            """
            This method solves the std form LP min (c.T * x) s.t. Ax = b, x >= 0 using primual-dual (infeasible) interior-point method.
            Parameters:
                c, A, b (np arrays): specify the LP in standard form
                epsilon        (float): duality gap threshold, specifies termination criteria
            Returns:
                x         (np array): solution to the LP
            """

            # ensure dimensions are okay
            assert A.shape[0] == b.shape[0], 'first dims of A and b must match, check input!'
            assert A.shape[1] == c.shape[0], 'second dim of A must match first dim of c, check input!'

            # ensure A is full rank, drop redundant rows if not
            if matrix_rank(A) < min(A.shape[0], A.shape[1]):
                print('A is not full rank, dropping redundant rows')
                _, pivots = sympy.Matrix(A).T.rref()
                A = A[list(pivots)]
                print('Shape of A after dropping redundant rows is {}'.format(A.shape))

            m = A.shape[0]
            n = A.shape[1]

            # initial solution (x_0, lambda_0, s_0) > 0 [lambda is variable l in code]
            # note that this is not a feasible solution in general
            # but it should tend towards feasibility by itself with iterations
            # therefore initially duality gap might show negative
            # since this is the infeasible-interior-point algorithm
            x = np.ones(shape=(n, ))
            l = np.ones(shape=(m, ))
            s = np.ones(shape=(n, ))

            # set iteration counter to 0 and mu_0
            k = 0

            # main loop body
            while abs(np.dot(x, s)) > epsilon:

                # print iteration number and progress
                k += 1
                primal_obj = np.dot(c, x)
                dual_obj = np.dot(b, l)
                print('iteration #{}; primal_obj = {:.5f}, dual_obj = {:.5f}; duality_gap = {:.5f}'.format(k, primal_obj, dual_obj, primal_obj - dual_obj))

                # choose sigma_k and calculate mu_k
                sigma_k = 0.4
                mu_k = np.dot(x, s) / n

                # create linear system A_ * delta = b_
                A_ = np.zeros(shape=(m + n + n, n + m + n))
                A_[0:m, 0:n] = np.copy(A)
                A_[m:m + n, n:n + m] = np.copy(A.T)
                A_[m:m + n, n + m:n + m + n] = np.eye(n)
                A_[m + n:m + n + n, 0:n] = np.copy(np.diag(s))
                A_[m + n:m + n + n, n + m:n + m + n] = np.copy(np.diag(x))

                b_ = np.zeros(shape=(n + m + n, ))
                b_[0:m] = np.copy(b - np.dot(A, x))
                b_[m:m + n] = np.copy(c - np.dot(A.T, l) - s)
                b_[m + n:m + n + n] = np.copy( sigma_k * mu_k * np.ones(shape=(n, )) - np.dot(np.dot(np.diag(x), np.diag(s)), np.ones(shape=(n, ))) )

                # solve for delta
                delta = np.linalg.solve(A_, b_)
                delta_x = delta[0:n]
                delta_l = delta[n:n + m]
                delta_s = delta[n + m:n + m + n]

                # find step-length alpha_k
                alpha_max = 1.0
                for i in range(n):
                    if delta_x[i] < 0:
                        alpha_max = min(alpha_max, -x[i]/delta_x[i])
                    if delta_s[i] < 0:
                        alpha_max = min(alpha_max, -s[i]/delta_s[i])
                eta_k = 0.99
                alpha_k = min(1.0, eta_k * alpha_max)

                # create new iterate
                x = x + alpha_k * delta_x
                l = l + alpha_k * delta_l
                s = s + alpha_k * delta_s

            # print difference between Ax and b
            diff = np.dot(A, x) - b
            print('Ax - b = {}; ideally it should have been zero vector'.format(diff))
            print('norm of Ax - b is = {}; ideally it should have been zero'.format(np.linalg.norm(diff)))

            return x

    solver = InteriorPointSolver()
    return solver.solve(c1, A1, b1)

A2 = np.array([[1, -1, 0]])
b2 = np.array([1])
c2 = np.array([1, -1, 1])
lp_solver_custom(c2, A2, b2)

x
[1. 1. 1.]
(1, 3)
x
[3.15111918e+16 3.15111918e+16 1.00000000e+00]
(1, 3)
x
[3.15111918e+16 3.15111918e+16 1.00000000e-01]
(1, 3)
x
[3.15111918e+16 3.15111918e+16 1.00000000e-02]
(1, 3)
x
[3.15111918e+16 3.15111918e+16 1.00000000e-03]
(1, 3)
x
[3.15111918e+16 3.15111918e+16 1.00000000e-04]
(1, 3)
x
[3.15111918e+16 3.15111918e+16 1.00000000e-05]
(1, 3)
x
[3.15111918e+16 3.15111918e+16 1.00000000e-06]
(1, 3)


array([3.15111918e+16, 3.15111918e+16, 1.00000000e-07])